<a href="https://colab.research.google.com/github/bphqk3/CF/blob/main/get_2122_game_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr  9 17:48:08 2024

@author: Brooke
"""

import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from io import StringIO

chromedriver_path = Service(r'C:\Users\Brooke\Desktop\chromedriver-win64\chromedriver-win64\chromedriver.exe')
coptions = webdriver.ChromeOptions()
coptions.add_experimental_option("excludeSwitches", ["enable-automation"])
coptions.add_argument("--enable-javascript")
coptions.add_argument("--window-size=1920,1080")
#coptions.add_argument('--headless')
coptions.add_argument('log-level=3')
coptions.add_argument('--disable-dev-shm-usage')
coptions.add_argument("--no-sandbox")
coptions.add_argument("--disable-crash-reporter")
coptions.add_argument("--disable-extensions")

player_url_list = ['https://www.nba.com/stats/players/boxscores-traditional?Season=2021-22',
                   'https://www.nba.com/stats/players/boxscores-advanced?Season=2021-22',
                   'https://www.nba.com/stats/players/boxscores-misc?Season=2021-22',
                   'https://www.nba.com/stats/players/boxscores-scoring?Season=2021-22',
                   'https://www.nba.com/stats/players/boxscores-usage?Season=2021-22']

team_url_list = ['https://www.nba.com/stats/teams/boxscores-traditional?Season=2021-22',
                 'https://www.nba.com/stats/teams/boxscores-advanced?Season=2021-22',
                 'https://www.nba.com/stats/teams/boxscores-scoring?Season=2021-22']

for link in player_url_list:
    url = link
    driver = webdriver.Chrome(service=chromedriver_path,options=coptions)
    driver.get(url)
    WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select")))
    dropdown = Select(driver.find_element(By.XPATH,"/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"))
    time.sleep(5)
    dropdown.select_by_visible_text('All')
    WebDriverWait(driver,20).until(EC.presence_of_element_located((By.CLASS_NAME,"Crom_table__p1iZz")))
    table_element = driver.find_element(By.CLASS_NAME, 'Crom_table__p1iZz')
    table_html = table_element.get_attribute('outerHTML')
    df = pd.read_html(StringIO(table_html))[0]

    df['GAME DATE']=pd.to_datetime(df['GAME DATE'], format='%m/%d/%Y')
    # Convert the date column to "yyyy-mm-dd" format
    df['GAME DATE'] = df['GAME DATE'].apply(lambda x: x.strftime('%Y-%m-%d'))

    unique_player = df['PLAYER'].unique().tolist()
    unique_player = sorted(unique_player)
    dff = pd.DataFrame(unique_player, columns=['PLAYER'])

    def generate_player_short(name, player_short_counts):
        name_parts = name.split(' ')
        first_initial = name_parts[0][0] + '.'
        player_short = first_initial + ' ' + ' '.join(name_parts[1:])
        if player_short in player_short_counts:
            player_short_counts[player_short] += 1
            return f"{player_short} {player_short_counts[player_short]}"
        else:
            player_short_counts[player_short] = 1
            return player_short
    # Add PLAYER_SHORT column
    player_short_counts = {}
    dff['Name'] = dff['PLAYER'].apply(lambda x: generate_player_short(x, player_short_counts))

    merged_df = pd.merge(dff,df,on='PLAYER',how='left')

    merged_df= merged_df.rename(columns={'PLAYER': 'Player', 'TEAM':'Team','MATCH UP':'Matchup','GAME DATE': 'Date'})
    merged_df['Pos']=[None] * len(df)
    merged_df.sort_values(by=['Date'], inplace = True)
    merged_df.reset_index(drop=True, inplace = True) #3/19/24
    stripped_url = url.split('-')[1]
    stripped_url = stripped_url.replace("?", "").replace("=", "")

    # Specify the full path to save the combined DataFrame
    output_directory = 'C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\nbacom\\game_history\\S2122\\'
    output_file_path = output_directory + 'player_stat_history' + stripped_url + '.parquet'

    # Save the combined DataFrame to the specified directory
    merged_df.to_parquet(output_file_path)
    driver.close()

for link in team_url_list:
    url = link
    driver = webdriver.Chrome(service=chromedriver_path,options=coptions)
    driver.get(url)
    WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,"/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select")))
    dropdown = Select(driver.find_element(By.XPATH,"/html/body/div[1]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select"))
    time.sleep(5)
    dropdown.select_by_visible_text('All')
    WebDriverWait(driver,20).until(EC.presence_of_element_located((By.CLASS_NAME,"Crom_table__p1iZz")))
    table_element = driver.find_element(By.CLASS_NAME, 'Crom_table__p1iZz')
    table_html = table_element.get_attribute('outerHTML')
    df = pd.read_html(StringIO(table_html))[0]

    if 'Game\xa0Date' in df.columns:
        # Rename the column to 'Game Date'
        df.rename(columns={'Team':'TEAM','Match\xa0Up':'MATCH UP','Game\xa0Date': 'GAME DATE'}, inplace=True)

    if 'Game Date' in df.columns:
        # Rename the column to 'Game Date'
        df.rename(columns={'Team':'TEAM','Match Up':'MATCH UP','Game Date': 'GAME DATE'}, inplace=True)

    if 'Matchup' in df.columns:
        # Rename the column to 'Game Date'
        df.rename(columns={'Team':'TEAM','Matchup':'MATCH UP','Date': 'GAME DATE'}, inplace=True)

    df['GAME DATE']=pd.to_datetime(df['GAME DATE'], format='%m/%d/%Y')
    # Convert the date column to "yyyy-mm-dd" format
    df['GAME DATE'] = df['GAME DATE'].apply(lambda x: x.strftime('%Y-%m-%d'))

    df= df.rename(columns={'TEAM':'Team','MATCH UP':'Matchup','GAME DATE': 'Date'})
    df.sort_values(by=['Date'], inplace = True)
    df.reset_index(drop=True,inplace = True) #3/19/24
    stripped_url = url.split('-')[1]
    stripped_url = stripped_url.replace("?", "").replace("=", "")
    # Specify the full path to save the combined DataFrame
    output_directory = 'C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\nbacom\\game_history\\S2122\\'
    output_file_path = output_directory + 'team_stat_history' + stripped_url + '.parquet'

    # Save the combined DataFrame to the specified directory
    df.to_parquet(output_file_path)
    driver.close()

#generate merged dataframe with all the stats you care about per PLAYER

a = pd.read_parquet('C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\nbacom\\game_history\\S2122\\player_stat_historytraditionalSeason2021.parquet')
a = a.drop(['STL','BLK','TOV','PF', '+/-'],axis=1)

b = pd.read_parquet('C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\nbacom\\game_history\\S2122\\player_stat_historyadvancedSeason2021.parquet')
b = b.drop(['W/L', 'MIN','AST/TO','OREB%', 'DREB%','REB%', 'TO\xa0Ratio','Pos'],axis=1)

merged_df = pd.merge(a,b, on=['Player', 'Name','Team','Matchup','Date'], how='inner')

c = pd.read_parquet('C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\nbacom\\game_history\\S2122\\player_stat_historymiscSeason2021.parquet')
c = c.drop(['W/L', 'MIN','Opp PTS\xa0OFF\xa0TO', 'Opp 2nd\xa0PTS', 'Opp FBPs','Opp PITP', 'BLK','BLKA','PF','Pos'],axis=1)
merged_df2 = pd.merge(merged_df,c, on=['Player', 'Name','Team','Matchup','Date'], how='inner')

d = pd.read_parquet('C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\nbacom\\game_history\\S2122\\player_stat_historyscoringSeason2021.parquet')
d = d.drop(['W/L', 'MIN','Pos'], axis=1)
merged_df3 = pd.merge(merged_df2,d, on=['Player', 'Name','Team','Matchup','Date'], how='inner')

e = pd.read_parquet('C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\nbacom\\game_history\\S2122\\player_stat_historyusageSeason2021.parquet')
e = e.drop(['W/L', 'MIN','Pos','USG%','%STL', '%BLK', '%BLKA'], axis=1)
merged_player_df = pd.merge(merged_df3,e, on=['Player', 'Name','Team','Matchup','Date'], how='inner')
merged_player_df.insert(4,'Opponent',merged_player_df['Matchup'].str[-3:])

f = pd.read_parquet('C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\nbacom\\game_history\\S2122\\team_stat_historytraditionalSeason2021.parquet')
#team stats become opponent allowed, forced, commited stats
f2 = f.drop(['W/L', 'MIN', 'STL', 'BLK', '+/-','Matchup', 'PTS', 'FGA', 'FG%', '3PA', 'FTA',
       'FT%','3P%','OREB', 'DREB', 'REB', 'AST', 'TOV','FGM','3PM', 'FTM'],axis = 1)
f2.rename(columns={'Team':'Opponent','PF':'Opp_Comitted_PF'}, inplace=True) #grab how many fouls team committed to add to row stat for opposing player
f3 = f.drop(['W/L', 'MIN','STL', 'BLK', '+/-','PF'],axis = 1)
f3.insert(0,'Opponent',f3['Matchup'].str[-3:])
new_names = [(i,'Opp_Allowed_'+i) for i in f3.iloc[:, 4:].columns.values]
f3.drop(['Team','Matchup'],axis=1,inplace = True)
f3.rename(columns = dict(new_names), inplace=True)
f3.rename(columns={'Opp_Allowed_TOV':'Opp_Forced_TOV'},inplace = True)
f3['Opp_Allowed_2PM'] = f3['Opp_Allowed_FGM'] - f3['Opp_Allowed_3PM']
f3['Opp_Allowed_2PA'] = f3['Opp_Allowed_FGA'] - f3['Opp_Allowed_3PA']
mf = pd.merge(f2,f3, on=['Opponent', 'Date'])

g = pd.read_parquet('C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\nbacom\\game_history\\S2122\\team_stat_historyadvancedSeason2021.parquet')
g = g.drop(['W/L', 'MIN', 'OffRtg', 'DefRtg', 'NetRtg','AST%', 'AST/TO', 'AST Ratio', 'eFG%','TS%', 'PIE'],axis=1)
g.insert(0,'Opponent',g['Matchup'].str[-3:])
g.drop(['Team','Matchup'],axis=1,inplace = True)
g.rename(columns={'OREB%':'Opp_Allowed_OREB%','DREB%':'Opp_Allowed_DREB%','REB%':'Opp_Allowed_REB%','TOV%':'Opp_Forced_TOV%','PACE':'Game Pace'},inplace = True)
mf2 = pd.merge(mf,g, on=['Opponent', 'Date'])

j = pd.read_parquet('C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\nbacom\\game_history\\S2122\\team_stat_historyscoringSeason2021.parquet')
j = j.drop(['W/L', 'MIN'],axis=1)
j.insert(0,'Opponent',j['Matchup'].str[-3:])
new_names = [(i,'Opp_Allowed_'+i) for i in j.iloc[:, 4:].columns.values]
j.drop(['Team','Matchup'],axis=1,inplace = True)
j.rename(columns = dict(new_names), inplace=True)
merged_opponent_team_df = pd.merge(mf2,j, on=['Opponent', 'Date'])

merged_player_w_opp_team_df = pd.merge(merged_player_df,merged_opponent_team_df, on=['Opponent', 'Date'])

#need to take non-percent team offensive stats to compute eventual player per 100 possessions stats
#also good to have TEAM offensive stats; note this is always the same as the Opp_Allowed stat
ff = pd.read_parquet('C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\nbacom\\game_history\\S2122\\team_stat_historytraditionalSeason2021.parquet')
team = ff.copy()
team = team.drop(['Matchup', 'W/L','MIN', 'FG%', '3P%', 'FT%', 'PF', '+/-'], axis = 1)
new_names = [(i,'TEAM_'+i) for i in team.iloc[:, 2:].columns.values]
team.rename(columns = dict(new_names), inplace=True)
merged_player_w_teamPopp_team_df = pd.merge(merged_player_w_opp_team_df,team, on = ['Team', 'Date'], how = 'outer')

#need to grab possession stats
url = 'https://www.pbpstats.com/games/nba?Season=2021-22&SeasonType=Regular%2BSeason'
driver = webdriver.Chrome(service=chromedriver_path,options=coptions)
driver.get(url)
WebDriverWait(driver,40).until(EC.presence_of_element_located((By.XPATH,"/html/body/div/div/main/div[3]/div/div/div[4]/div[1]/form/select")))
pgsel = Select(driver.find_element(By.XPATH,"/html/body/div/div/main/div[3]/div/div/div[4]/div[1]/form/select"))
time.sleep(5)
pgsel.select_by_visible_text('All')
WebDriverWait(driver,40).until(EC.presence_of_element_located((By.CLASS_NAME,"vgt-responsive")))
table_element = driver.find_element(By.CLASS_NAME, 'vgt-responsive')
table_html = table_element.get_attribute('outerHTML')
poss_df = pd.read_html(StringIO(table_html))[0]
driver.close()
#drop points cols
poss_df = poss_df.drop(poss_df.columns[[3, 4]], axis=1)

poss_df.rename(columns={poss_df.columns[0]: 'Date',
                        poss_df.columns[1]: 'Home',
                        poss_df.columns[2]: 'Away',
                        poss_df.columns[3]: 'Home Possessions',
                        poss_df.columns[4]:'Away Possessions' }, inplace = True)

# Convert back to string in "Y-M-D" format
poss_df['Date'] = poss_df['Date'].apply(lambda x: pd.to_datetime(x).strftime("%Y-%m-%d"))
df1 = poss_df.copy()
df2 = poss_df.copy()
df1['Team'] = df1.apply(lambda row: row['Away'], axis=1)
df1['TEAM_Poss'] = df1.apply(lambda row: row['Away Possessions'], axis=1)
df1['Opp_Poss'] = df1.apply(lambda row: row['Home Possessions'], axis=1)
df2['Team'] = df1.apply(lambda row: row['Home'], axis=1)
df2['TEAM_Poss'] = df2.apply(lambda row: row['Home Possessions'], axis=1)
df2['Opp_Poss'] = df2.apply(lambda row: row['Away Possessions'], axis=1)
poss_df = pd.concat([df1,df2],ignore_index=True)
poss_df = poss_df.drop(['Home', 'Away', 'Home Possessions', 'Away Possessions'], axis = 1)
merged_player_stats_wteam_wopp_team_df = pd.merge(merged_player_w_teamPopp_team_df,poss_df, on = ['Team', 'Date'], how = 'outer')
merged_player_stats_wteam_wopp_team_df.sort_values(by=['Date','Name'], inplace = True)
merged_player_stats_wteam_wopp_team_df.reset_index(drop=True,inplace = True)

s = merged_player_stats_wteam_wopp_team_df

output_directory = 'C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\nbacom\\game_history\\S2122\\'
output_file_path = output_directory + 'raw_full_stats' + '.parquet'

# Save the combined DataFrame to the specified directory
s.to_parquet(output_file_path)



s = pd.read_parquet('C:/Users/Brooke/Desktop/rbt/rbt_player_history/nbacom/game_history/S2122/raw_full_stats.parquet')
#s['P+A'] = s['PTS'] + s['AST']
#s['P+R'] = s['PTS'] + s['REB']
#s['P+R+A'] = s['PTS'] + s['REB'] + s['AST']
#s['R+A'] = s['REB'] + s['AST']

s = s.drop(['OREB','DREB','REB','AST'],axis=1)

s['Total_Game_Poss'] = s.groupby("Name").apply(lambda x: x["Opp_Poss"] + x["TEAM_Poss"]).reset_index(level=0, drop=True)

s['2PM'] = s.groupby("Name").apply(lambda x: x['FGM'] - x['3PM']).reset_index(level=0, drop=True)
s['2PA'] = s.groupby("Name").apply(lambda x: x['FGA'] - x['3PA']).reset_index(level=0, drop=True)


#totals need to be on a per player basis because players don't play every game
#this means that true team totals would be the cumsum of the Team + unique(Date) columns
ts = s[["Name","Team","Date","TEAM_Poss","TEAM_PTS"]]
ts['Total_TEAM_Poss'] = ts.groupby(["Name","Team"], group_keys=False)['TEAM_Poss'].cumsum()
ts['Total_TEAM_PTS'] = ts.groupby(["Name","Team"], group_keys=False)['TEAM_PTS'].cumsum()

s = pd.merge(s,ts,on=["Name","Team","Date","TEAM_Poss","TEAM_PTS"],how='left')

s['Total_PTS'] = s.groupby("Name", group_keys=False)['PTS'].cumsum()
s['Total_%TEAM_PTS'] = round((s['Total_PTS']/s['Total_TEAM_PTS'])*100,2)
#s['Total_FGM'] = s.groupby("Name", group_keys=False)['FGM'].cumsum()
s = s.drop(['FGM'],axis=1)
s['Total_FGA'] = s.groupby("Name", group_keys=False)['FGA'].cumsum()

s['Total_2PM'] = s.groupby("Name", group_keys=False)['2PM'].cumsum()
s['Total_2PA'] = s.groupby("Name", group_keys=False)['2PA'].cumsum()
s['Total_2P%'] = round((s['Total_2PM'] / s['Total_2PA'])*100,2)

s['Total_3PM'] = s.groupby("Name", group_keys=False)['3PM'].cumsum()
s['Total_3PA'] = s.groupby("Name", group_keys=False)['3PA'].cumsum()
s['Total_3P%'] = round((s['Total_3PM'] / s['Total_3PA'])*100,2)

s['Total_FTM'] = s.groupby("Name", group_keys=False)['FTM'].cumsum()
s['Total_FTA'] = s.groupby("Name", group_keys=False)['FTA'].cumsum()
s['Total_FT%'] = round((s['Total_FTM'] / s['Total_FTA'])*100,2)
#Points/ [2*(Field Goals Attempted+0.44*Free Throws Attempted)]
s['Total_TS%'] = round(s['Total_PTS'] / (2 * ((s['Total_FGA']) + (0.44 * s['Total_FTA'])))*100,2)
#s['Total_OREB'] = s.groupby("Name", group_keys=False)['OREB'].cumsum()
#s['Total_DREB'] = s.groupby("Name", group_keys=False)['DREB'].cumsum()
#s['Total_REB'] = s.groupby("Name", group_keys=False)['REB'].cumsum()


#opponent stats should be true team stats against the player
#TEAM_Poss = Opp_Allowed_Team_Poss
os = s[["Opponent","Date","Opp_Poss","TEAM_Poss", "Total_Game_Poss","Opp_Allowed_PTS","Opp_Allowed_FGA",
        "Opp_Allowed_2PM","Opp_Allowed_2PA","Opp_Allowed_3PM","Opp_Allowed_3PA",
        "Opp_Allowed_FTM","Opp_Allowed_FTA","Opp_Allowed_REB","Opp_Forced_TOV"]]

os.rename(columns={'TEAM_Poss':'Opp_Allowed_Team_Poss'},inplace=True)
os = os.drop_duplicates()

os['Total_Opp_Poss'] = os.groupby(["Opponent"], group_keys=False)['Opp_Poss'].cumsum()
os['Total_Opp_Allowed_Team_Poss'] = os.groupby(["Opponent"], group_keys=False)['Opp_Allowed_Team_Poss'].cumsum()
os['Total_Opp_Total_Game_Poss'] = os.groupby(["Opponent"], group_keys=False)['Total_Game_Poss'].cumsum()

#os['Total_Opp_Comitted_PF'] = os.groupby(["Opponent"], group_keys=False)['Opp_Comitted_PF'].cumsum()
os['Total_Opp_Allowed_PTS'] = os.groupby(["Opponent"], group_keys=False)['Opp_Allowed_PTS'].cumsum()
os['Total_Opp_Allowed_FGA'] = os.groupby(["Opponent"], group_keys=False)['Opp_Allowed_FGA'].cumsum()

os['Total_Opp_Allowed_2PM'] = os.groupby(["Opponent"], group_keys=False)['Opp_Allowed_2PM'].cumsum()
os['Total_Opp_Allowed_2PA'] = os.groupby(["Opponent"], group_keys=False)['Opp_Allowed_2PA'].cumsum()
os['Total_Opp_Allowed_2P%'] = round(os['Total_Opp_Allowed_2PM']/os['Total_Opp_Allowed_2PA']*100,2)

os['Total_Opp_Allowed_3PM'] = os.groupby(["Opponent"], group_keys=False)['Opp_Allowed_3PM'].cumsum()
os['Total_Opp_Allowed_3PA'] = os.groupby(["Opponent"], group_keys=False)['Opp_Allowed_3PA'].cumsum()
os['Total_Opp_Allowed_3P%'] = round(os['Total_Opp_Allowed_3PM'] / os['Total_Opp_Allowed_3PA']*100,2)

os['Total_Opp_Allowed_FTM'] = os.groupby(["Opponent"], group_keys=False)['Opp_Allowed_FTM'].cumsum()
os['Total_Opp_Allowed_FTA'] = os.groupby(["Opponent"], group_keys=False)['Opp_Allowed_FTA'].cumsum()
os['Total_Opp_Allowed_FT%'] = round(os['Total_Opp_Allowed_FTM']/os['Total_Opp_Allowed_FTA']*100,2)

os['Total_Opp_Allowed_REB'] = os.groupby(["Opponent"], group_keys=False)['Opp_Allowed_REB'].cumsum()
os['Total_Opp_Forced_TOV'] = os.groupby(["Opponent"], group_keys=False)['Opp_Forced_TOV'].cumsum()

s = pd.merge(s,os,on=["Opponent","Date","Opp_Poss", "Total_Game_Poss","Opp_Allowed_PTS","Opp_Allowed_FGA",
        "Opp_Allowed_2PM","Opp_Allowed_2PA","Opp_Allowed_3PM","Opp_Allowed_3PA",
        "Opp_Allowed_FTM","Opp_Allowed_FTA","Opp_Allowed_REB","Opp_Forced_TOV"],how='left')

# this will allow me to take rolling stats with each day
s['PTS_per100'] = round(s['Total_PTS']*100/s['Total_TEAM_Poss'],2)
s['2PA_per100'] = round(s['Total_2PA']*100/s['Total_TEAM_Poss'],2)
s['3PA_per100'] = round(s['Total_3PA']*100/s['Total_TEAM_Poss'],2)
s['FTA_per100'] = round(s['Total_FTA']*100/s['Total_TEAM_Poss'],2)
#s['PFD_per100'] = round(s['Total_PFD']*100/s['Total_TEAM_Poss'],2)

#think on whether or not you want to take a differential of these stats as new columns?
#currently think the rolling stats have this covered


#s['OREB_per100'] = round(s['OREB']*100/s['TEAM_Poss'],2)
#s['DREB_per100'] = round(s['DREB']*100/s['TEAM_Poss'],2)
#s['REB_per100'] = round(s['REB']*100/s['TEAM_Poss'],2)
#s['AST_per100'] =round(s['AST']*100/s['TEAM_Poss'],2)
#s['PTS\xa0OFF\xa0TO_per100'] = round(s['PTS\xa0OFF\xa0TO']*100/s['TEAM_Poss'],2)
#s['2nd\xa0PTS_per100']= round(s['2nd\xa0PTS']*100/s['TEAM_Poss'],2)
#s['FBPs_per100'] = round(s['FBPs']*100/s['TEAM_Poss'],2)
#s['PITP_per100']= round(s['PITP']*100/s['TEAM_Poss'],2)
s = s.drop(['PTS\xa0OFF\xa0TO', '2nd\xa0PTS', 'FBPs', 'PITP'],axis=1)

#s['Opp_Comitted_PF_per100']= round(s['Total_Opp_Comitted_PF']*100/(s['Total_Opp_Total_Game_Poss']),2)
s['Opp_Allowed_PTS_per100'] = round(s['Total_Opp_Allowed_PTS']*100/s['Total_Opp_Allowed_Team_Poss'],2)
s['Opp_Allowed_2PA_per100'] = round(s['Total_Opp_Allowed_2PA']*100/s['Total_Opp_Allowed_Team_Poss'],2)
s['Opp_Allowed_3PA_per100'] = round(s['Total_Opp_Allowed_3PA']*100/s['Total_Opp_Allowed_Team_Poss'],2)
s['Opp_Allowed_FTA_per100'] = round(s['Total_Opp_Allowed_FTA']*100/s['Total_Opp_Allowed_Team_Poss'],2)
#s['Opp_Allowed_OREB_per100'] = round(s['Opp_Allowed_OREB']*100/s['TEAM_Poss'],2)
#s['Opp_Allowed_DREB_per100'] = round(s['Opp_Allowed_DREB']*100/s['TEAM_Poss'],2)
s['Opp_Allowed_REB_per100'] = round(s['Total_Opp_Allowed_REB']*100/s['Total_Opp_Allowed_Team_Poss'],2)
s['Opp_Forced_TOV_per100'] = round(s['Total_Opp_Forced_TOV']*100/s['Total_Opp_Allowed_Team_Poss'],2)


s['Date'] = pd.to_datetime(s['Date'])
s['Days_Rest'] = s.groupby("Name",group_keys=False)['Date'].diff().dt.days
s['Date']=s['Date'].dt.date
columns_to_move1 = s.pop('Days_Rest')
s.insert(loc=6, column='Days_Rest', value=columns_to_move1)
s['Days_Rest'] = s['Days_Rest'].fillna(0)

val1 = s.copy()

# cols_avg = ['P+A', 'P+R','R+A',
#             'P+R+A']


#  ### This function changes each stat to be the median of the window_size num_games for each team,
### and shifts it one so it does not include the current stats from box score


player_med_cols = ['MIN', 'PTS_per100', '2PA_per100', 'Total_2P%', '3PA_per100', 'Total_3P%',
            'FTA_per100', 'Total_FT%', 'Total_TS%', 'PACE', '%PTS 2PT',
            '%PTS 3PT', '%PTS FT', 'Total_%TEAM_PTS']

def transform_and_shift_gb_name(df, player_med_cols):
    for column in player_med_cols:
        for window_size in [1, 5, 10, 20]:
            new_col_name = f'{column}_rolling_med{window_size}'
            if window_size == 1:
                rolling_med = df.groupby("Name")[column].transform(lambda x: x.rolling(window=len(x), min_periods=window_size).median().round(2))
            else:
                rolling_med = df.groupby("Name")[column].transform(lambda x: x.rolling(window=window_size, min_periods=window_size).median().round(2))
            df[new_col_name] = rolling_med.groupby(df["Name"]).shift(1)

    return df

# Apply the transformation and shifting within each group for each column in cols_med
s = transform_and_shift_gb_name(s, player_med_cols)


team_med_cols = ["TEAM_Poss","TEAM_PTS"]

ts2 = s[["Team","Date","TEAM_Poss","TEAM_PTS"]]
ts2 = ts2.drop_duplicates()


def transform_and_shift_gb_team(df, team_med_cols):
    for column in team_med_cols:
        for window_size in [1, 5, 10, 20]:
            new_col_name = f'{column}_rolling_med{window_size}'
            if window_size == 1:
                rolling_med = df.groupby("Team")[column].transform(lambda x: x.rolling(window=len(x), min_periods=window_size).median().round(2))
            else:
                rolling_med = df.groupby("Team")[column].transform(lambda x: x.rolling(window=window_size, min_periods=window_size).median().round(2))
            df[new_col_name] = rolling_med.groupby(df["Team"]).shift(1)

    return df

ts2 = transform_and_shift_gb_team(ts2, team_med_cols)
s = pd.merge(s,ts2,on=["Team","Date","TEAM_Poss","TEAM_PTS"],how='left')


opp_med_cols = ['Opp_Poss','Opp_Allowed_Team_Poss','Opp_Allowed_PTS_per100','Opp_Allowed_2PA_per100',
            'Opp_Allowed_3PA_per100','Opp_Allowed_FTA_per100','Opp_Allowed_REB_per100',
            'Opp_Forced_TOV_per100','Total_Opp_Allowed_2P%','Total_Opp_Allowed_3P%',
            'Total_Opp_Allowed_FT%','Game Pace']

os2 = s[['Opponent','Date','Opp_Poss','Opp_Allowed_Team_Poss',
         'Opp_Allowed_PTS_per100','Opp_Allowed_2PA_per100',
            'Opp_Allowed_3PA_per100','Opp_Allowed_FTA_per100','Opp_Allowed_REB_per100',
            'Opp_Forced_TOV_per100','Total_Opp_Allowed_2P%','Total_Opp_Allowed_3P%',
            'Total_Opp_Allowed_FT%','Game Pace']]

os2 = os2.drop_duplicates()

def transform_and_shift_gb_opp(df, opp_med_cols):
    for column in opp_med_cols:
        for window_size in [1, 5, 10, 20]:
            new_col_name = f'{column}_rolling_med{window_size}'
            if window_size == 1:
                rolling_med = df.groupby("Opponent")[column].transform(lambda x: x.rolling(window=len(x), min_periods=window_size).median().round(2))
            else:
                rolling_med = df.groupby("Opponent")[column].transform(lambda x: x.rolling(window=window_size, min_periods=window_size).median().round(2))
            df[new_col_name] = rolling_med.groupby(df["Opponent"]).shift(1)

    return df


os2 = transform_and_shift_gb_opp(os2, opp_med_cols)
s = pd.merge(s,os2,on=['Opponent','Date','Opp_Poss','Opp_Allowed_Team_Poss','Opp_Allowed_PTS_per100','Opp_Allowed_2PA_per100',
            'Opp_Allowed_3PA_per100','Opp_Allowed_FTA_per100','Opp_Allowed_REB_per100',
            'Opp_Forced_TOV_per100','Total_Opp_Allowed_2P%','Total_Opp_Allowed_3P%',
            'Total_Opp_Allowed_FT%','Game Pace'],how='left')



output_directory = 'C:\\Users\\Brooke\\Desktop\\rbt\\rbt_player_history\\S2122\\'
output_file_path = output_directory + 'player_full_stat_history.parquet'
s.to_parquet(output_file_path)
